In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df=pd.read_csv("Employers_data.csv")
df.shape

(10000, 10)

In [3]:
df=df.drop(columns=['Employee_ID','Name'])

In [4]:
df.columns

Index(['Age', 'Gender', 'Department', 'Job_Title', 'Experience_Years',
       'Education_Level', 'Location', 'Salary'],
      dtype='object')

In [5]:
df= pd.get_dummies(df, columns=['Location', 'Gender','Department','Job_Title','Education_Level'], drop_first=True)


In [6]:
m1=df['Age'].mean()
d1=df['Age'].std()
m2=df['Experience_Years'].mean()
d2=df['Experience_Years'].std()
m3=df['Salary'].mean()
d3=df['Salary'].std()

In [7]:
df

,Age,Experience_Years,Salary,Location_Chicago,Location_New York,Location_San Francisco,Location_Seattle,Gender_Male,Department_Finance,Department_HR,Department_Marketing,Department_Product,Department_Sales,Job_Title_Engineer,Job_Title_Executive,Job_Title_Intern,Job_Title_Manager,Education_Level_Master,Education_Level_PhD
0,24,1,90000,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False
1,56,33,195000,False,False,False,True,True,False,False,False,False,True,False,True,False,False,True,False
2,21,1,35000,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False
3,30,9,75000,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False
4,25,2,70000,False,False,False,True,False,False,True,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,51,28,185000,False,False,False,False,True,False,False,False,False,True,False,True,False,False,True,False
9996,36,9,165000,False,False,True,False,False,False,False,False,False,True,False,True,False,False,False,True
9997,57,30,200000,False,True,False,False,True,False,False,False,False,True,False,True,False,False,False,True
9998,37,14,135000,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False


In [8]:
df['normalage']=(df['Age']-m1)/d1

In [9]:
df['normalexp']=(df['Experience_Years']-m2)/d2

In [10]:
df['normalsalary']=(df['Salary']-m3)/d3

In [11]:
y=df['normalsalary']

In [12]:
x=df[['Location_Chicago','Location_New York', 'Location_San Francisco', 'Location_Seattle',
       'Gender_Male', 'Department_Finance', 'Department_HR',
       'Department_Marketing', 'Department_Product', 'Department_Sales',
       'Job_Title_Engineer', 'Job_Title_Executive', 'Job_Title_Intern',
       'Job_Title_Manager', 'Education_Level_Master', 'Education_Level_PhD',
       'normalage', 'normalexp']]

In [13]:
xtrain,xtemp,ytrain,ytemp=train_test_split(x,y, test_size=0.30, random_state=42)

In [14]:
xtest,xval,ytest,yval=train_test_split(xtemp,ytemp,test_size=0.33,random_state=42)

In [15]:
xtrain = xtrain.values        # shape (7000, 18)
ytrain = ytrain.values.reshape(-1, 1)

In [16]:
ytrain

array([[-0.87659832],
       [ 0.4258768 ],
       [-0.98513791],
       ...,
       [ 0.86003518],
       [-1.0936775 ],
       [ 0.5344164 ]], shape=(7000, 1))

In [17]:
def ReLU(x):
    return np.maximum(0,x)

first layer

In [43]:
weight0to1=np.random.normal(0,math.sqrt(2/18),(18,256))
bias0to1=np.full((256),0.01)

In [44]:
forward0to1=np.dot(xtrain,weight0to1)+bias0to1

In [45]:
forward0to1.shape

(7000, 256)

In [46]:
reluactivated1=ReLU(forward0to1)

In [47]:
reluactivated1.shape

(7000, 256)

second layer 

In [48]:
weight1to2=np.random.normal(0,math.sqrt(2/256),(256,32))  
bias1to2=np.full(32,(0.01))


In [49]:
forward1to2=np.dot(reluactivated1,weight1to2)+bias1to2

In [50]:
reluactivated2=ReLU(forward1to2)

final layer

In [51]:
weight2to3=np.random.normal(0,math.sqrt(2/32),(32,1)) 
bias2to3=np.full(1,(0.01))

In [52]:
forward2to3=np.dot(reluactivated2,weight2to3)+bias2to3

In [53]:
mean_squared_error(forward2to3,ytrain)

3.1397546800823943

backpropogation

In [54]:
lastgradarr=np.empty((7000,1))
for i in range(7000):
    lastgradarr[i][0]=-2*(ytrain[i][0]-forward2to3[i][0])
lastgrad=np.sum(lastgradarr)/7000


In [55]:
arr=[forward0to1.shape,forward1to2.shape,forward2to3.shape]
arr2=[reluactivated1.shape,reluactivated2.shape]
print(arr)
arr2

[(7000, 256), (7000, 32), (7000, 1)]


[(7000, 256), (7000, 32)]

In [56]:
secondlastgrad=np.zeros(32)
secondlastgrad = secondlastgrad.reshape(-1, 1)


In [57]:
for i in range(7000):
    for j in range(32):
        secondlastgrad[j]+=lastgradarr[i][0]*reluactivated2[i][j]
for i in range(32):
    secondlastgrad[i]=secondlastgrad[i]/7000

In [58]:
savedgrad=np.zeros((7000,32))
for i in range (7000):
    tempar=np.zeros(32)
    for j in range(32):
        tempar[j]=lastgradarr[i][0]*weight2to3[j][0]
        if(forward1to2[i][j]>0):
            savedgrad[i][j]=tempar[j]
        else:
            savedgrad[i][j]=0

In [59]:
thirdlastgradarr=np.zeros((256,32))
for i in range (7000):
    for j in range(256):
        for k in range(32):
            thirdlastgradarr[j][k]+=reluactivated1[i][j]*savedgrad[i][k]
for j in range(256):
    for k in range(32):
        thirdlastgradarr[j][k]=thirdlastgradarr[j][k]/7000

In [60]:
savedgrad2=np.zeros((7000,256))
for i in range (7000):
    for j in range(256):
        for k in range(32):
            savedgrad2[i][j]+=weight1to2[j][k]*savedgrad[i][k]
        if(forward0to1[i][j]>0):
            savedgrad2[i][j]=savedgrad2[i][j]
        else:
            savedgrad2[i][j]=0

In [61]:
savedgrad2.shape

(7000, 256)

In [62]:
fourthlastgradarr=np.zeros((18,256))
for i in range(7000):
    for j in range(18):
        for k in range(256):
            fourthlastgradarr[j][k]+=savedgrad2[i][k]*xtrain[i][j]
for i in range(18):
    for j in range (256):
        fourthlastgradarr[i][j]=fourthlastgradarr[i][j]/7000


bias gradient 

In [63]:
biasgrad1=lastgrad
biasgrad2=np.zeros(32)
for i in range(7000):
    for j in range(32):
        biasgrad2[j]+=savedgrad[i][j]

In [64]:
biasgrad3=np.zeros(256)
for i in range(7000):
    for j in range(256):
        biasgrad3[j]+=savedgrad2[i][j]

In [65]:
for i in range(32):
    biasgrad2[i]=biasgrad2[i]/7000
for i in range(256):
    biasgrad3[i]=biasgrad3[i]/7000

update step

In [ ]:
def updater(param, grad, lr=0.001):
    if param.ndim == 1:
        for i in range(param.shape[0]):
            param[i] -= lr * grad[i]
    else:
        for i in range(param.shape[0]):
            for j in range(param.shape[1]):
                param[i][j] -= lr * grad[i][j]

In [68]:
def update():
    updater(weight0to1, fourthlastgradarr)
    updater(weight1to2, thirdlastgradarr)
    updater(weight2to3, secondlastgrad)

    updater(bias0to1, biasgrad3)
    updater(bias1to2, biasgrad2)

    bias2to3[0] -= 0.001 * biasgrad1
